<a href="https://colab.research.google.com/github/Daniil81704/Fine-tuning-w-LoRA/blob/main/Dostoevskiy_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -U datasets evaluate peft

In [1]:
import requests

url = "https://gitlab.com/z00logist/artificial-dostoevsky/-/raw/main/data/corpus.txt"
try:
    response = requests.get(url)
    response.raise_for_status()
    text_from_url = response.text
    # print(text_from_url)
except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе URL: {e}")

In [2]:
text_from_url[:1000]

'Ох уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь... невольно задумаешься, а там всякая дребедень и пойдет в голову; право бы, запретил им писать; так-таки просто вовсе бы запретил.Кн. В Ф. Одоевский Апреля .  Бесценная моя Варвара Алексеевна! Вчера я был счастлив, чрезмерно счастлив, донельзя счастлив! Вы хоть раз в жизни, упрямица, меня послушались. Вечером, часов в восемь, просыпаюсь (вы знаете, маточка, что я часочек-другой люблю поспать после должности), свечку достал, приготовляю бумаги, чиню перо, вдруг, невзначай, подымаю глаза, право, у меня сердце нот так и запрыгало! Так вы-таки поняли, чего мне хотелось, чего сердчишку моему хотелось! Вижу, уголочек занавески у окна вашего загнут и прицеплен к горшку с бальзамином, точнехонько так, как я вам тогда намекал; тут же показалось мне, что и личико ваше мелькнуло у окна, что и вы ко мне из

In [3]:
import torch
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig

def preprocess_function(examples):
    encoding = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=50)
    encoding['labels'] = encoding['input_ids'].copy()
    return encoding

In [9]:
model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = torch.device('cpu')
print(DEVICE)
model.to(DEVICE)

cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=4608, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=1536)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6144, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=6144)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

In [10]:
data = {
    "text": [text_from_url]
}

dataset = Dataset.from_dict(data)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

lora_config = LoraConfig(
    r=64,
    lora_alpha=64,
    lora_dropout=0.1,
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    report_to='none',
    learning_rate=5e-5,
    num_train_epochs=800,
    per_device_train_batch_size=8,
    warmup_steps=10,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

# model.save_pretrained('./finetuned_gpt3')

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,0.497700


TrainOutput(global_step=800, training_loss=0.31389128267765043, metrics={'train_runtime': 69.3709, 'train_samples_per_second': 11.532, 'train_steps_per_second': 11.532, 'total_flos': 165455216640000.0, 'train_loss': 0.31389128267765043, 'epoch': 800.0})

In [11]:
import numpy as np

np.random.seed(42)
torch.manual_seed(42)

text = "Не хотите ли кофею?"
inpt = tokenizer.encode(text, return_tensors="pt")

out = model.generate(inpt.cuda(),
                     max_length=200,
                     repetition_penalty=6.0,
                     do_sample=True,
                     top_k=5,
                     top_p=0.95,
                     temperature=1,
                     no_repeat_ngram_size=2)

generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Не хотите ли кофею?
— Нет, благодарю вас. Я пью чай с лимоном и медом: очень вкусно! — Он засмеялся; но тотчас же спохватился он в эту минуту своей шутки.— А вы не хотели бы чаю выпить у меня на дворе под яблоней?.. У нас есть там уголок для всех наших домашних животных,— я их прикармливаю каждый день по полтине серебром за ведро... Хотите быть моим домашним животным?- спросил вдруг старик.- Да нет!.. Зачем мне это нужно?! Не стоит утруждаться вашим обществом... Но если вам так будет угодно...- Старик опять захохотал весело-весело.... "Ах ты боже мой!- подумал про себя Павел Петрович,- что со мной сталось?" И точно сказать ему было трудно -- как будто все исчезло разом из памяти его." 
 В то время когда Пушкин писал эти стихи (в Михайловском), поэт был уже болен лихорадкой"."Он лежал больной почти два месяца без сознания между тем припадком горячки",-- пишет П
